In [302]:
import sys
sys.path.append('../src')
# from causal_shapley import causal_shapley, predict_proba
from egtoolkit import *
%load_ext autoreload
%autoreload 2

from egtoolkit import *

import random
import warnings
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import itertools
from itertools import combinations, permutations, chain

import shap

warnings.filterwarnings("ignore")
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.under_sampling import RandomUnderSampler

from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete.CPD import TabularCPD

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Import dataset

In [303]:
categorical_cols = ['global_active_power', 'global_reactive_power', 'voltage',
                    'global_intensity', 'kitchen', 'laundry', 'climate_control', 'other',
                    'weekend', 'month_name', 'season_name', 'day_name']  # list all columns that are categorical

In [304]:
dataset = pd.read_csv('./datasets/2.0-discretized-v2-3-peak-label-encoded.csv')

In [305]:
dataset[['peak_warning']].values

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [306]:
targets = ['peak_label_pred', 'peak_warning', 'no_significant_change', 'lower_than_usual']
targets_only = ['peak_label_pred', 'no_significant_change', 'lower_than_usual']

In [307]:
dataset.columns

Index(['global_active_power', 'global_reactive_power', 'voltage',
       'global_intensity', 'kitchen', 'laundry', 'climate_control', 'other',
       'weekend', 'month_name', 'season_name', 'day_name', 'peak_label_pred',
       'peak_warning', 'no_significant_change', 'lower_than_usual'],
      dtype='object')

In [308]:
dataset['peak_warning'].values

array([False, False, False, ..., False, False, False])

In [309]:
dataset.shape

(41673, 16)

In [310]:
sample = 300
background = dataset.drop(columns=targets).sample(sample, random_state=0)
indexes = background.index.tolist()

In [311]:
dataset.iloc[indexes]

,global_active_power,global_reactive_power,voltage,global_intensity,kitchen,laundry,climate_control,other,weekend,month_name,season_name,day_name,peak_label_pred,peak_warning,no_significant_change,lower_than_usual
35784,Low,Medium,High,Low,Medium,High,Low,Medium,False,november,autumn,thursday,lower_than_usual,False,False,True
8217,Low,Low,High,Low,High,Medium,Medium,Low,True,october,autumn,saturday,no_significant_change,False,True,False
13032,Medium,Low,Very High,Medium,Medium,Medium,Low,High,False,december,winter,thursday,no_significant_change,False,True,False
6450,High,High,Very High,High,Low,High,Very High,High,True,april,spring,sunday,no_significant_change,False,True,False
36805,High,Low,Very High,High,Medium,Medium,High,Low,True,january,winter,saturday,lower_than_usual,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13035,Low,Low,High,Low,Very High,High,High,Low,False,june,summer,monday,no_significant_change,False,True,False
14078,Very High,Very High,Low,Very High,High,Very High,Very High,High,True,february,winter,sunday,peak_warning,True,False,False
37704,Low,Medium,Medium,Low,Low,High,Low,Medium,False,february,winter,friday,lower_than_usual,False,False,True
2616,Medium,Low,Low,Medium,High,Medium,Medium,High,False,march,spring,friday,no_significant_change,False,True,False


In [318]:
def custom_predict(data):
    encode = {True: 1.0, False: 0.0}
    # data = dataset.drop(columns=targets_only).iloc[indexes].values
    # print(data2 == data)/
    # print(data[:,-1])
    
    # print(type(data) == np.ndarray)
    if (len(data) > 1):
        print('banyak')
        data = dataset.drop(columns=targets_only).iloc[indexes].values
        # print(data)
        pred = [encode[p] for p in data[:,-1]]
        pred = np.array(pred).reshape(-1, 1) #np.array(pred)
        
    else:
        print('dikit')
        data = dataset.drop(columns=targets_only).iloc[0].values
        print('data', data[-1]) 
        pred = np.array(encode[data[-1]]).reshape(-1, 1)
        print('encode',[pred])
        
        
    # print(data)
    # if isinstance(data, pd.DataFrame):
    #     print('data columns pandas')
    #     print(data.columns)
    #     pred = [encode[p] for p in data['peak_warning'].values]
    # else:
    #     pred = [encode[p] for p in data[:, -1]]
    #     print('else')
    #     print(data[:, -1])
        
    print('output size', pred.shape)
    # print(pred)
    
    return pred


# background = dataset.drop(columns=targets).sample(sample, random_state=0)  # Background data sample
explainer = shap.KernelExplainer(custom_predict, background)
background.shape, observation.shape


banyak
output size (300, 1)


((300, 12), (1, 12))

In [319]:
observation = dataset.drop(columns=targets).iloc[0:1]
print("Observation shape:", observation.shape)  
# observation
# shap_values = explainer.shap_values(observation)

# print(shap_values)
try:
    shap_values = explainer.shap_values(observation)
except Exception as e:
    print("Error computing SHAP values:", e)

Observation shape: (1, 12)


  0%|          | 0/1 [00:00<?, ?it/s]INFO:shap:num_full_subsets = 3
INFO:shap:remaining_weight_vector = array([0.35673839, 0.32616082, 0.31710079])
INFO:shap:num_paired_subset_sizes = 5
INFO:shap:weight_left = 0.2929005745959313
  0%|          | 0/1 [00:00<?, ?it/s]

dikit
data False
encode [array([[0.]])]
output size (1, 1)
banyak
output size (300, 1)
Error computing SHAP values: cannot reshape array of size 300 into shape (621600,1)
